# 🧠 Python 考试参考讲义：带函数讲解与注释代码
本 Notebook 汇总了讲解内容与完整注释代码，可直接在 Jupyter Notebook 中运行。

## 📚 目录
1. 文件头与导入库
2. 数据读取与清洗（pandas）
3. 可视化（matplotlib）
4. 数据库存储与查询（sqlite3）
5. 神经网络实现与训练（PyTorch）

## 📘 1. 文件头与导入库
- 指定 UTF-8 编码，防止中文乱码。
- 导入 pandas、matplotlib、sqlite3、torch 等模块：用于数据处理、可视化、数据库操作与神经网络。
- 通过 `os.environ` 设置 `KMP_DUPLICATE_LIB_OK`（临时解决 OpenMP 冲突，仅救急使用）。

In [ ]:
# -*- coding: utf-8 -*-  # 指定源码文件编码，防止中文乱码
import pandas as pd                   # 数据处理
import matplotlib.pyplot as plt       # 绘图
import sqlite3                        # SQLite 数据库
import torch                          # PyTorch 张量运算
import torch.nn as nn                 # 神经网络模块
import torch.optim as optim           # 优化器
import os                             # 操作系统接口

# 临时允许重复 OpenMP 运行时继续执行（长期不推荐）
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

## 📊 2. 数据读取与清洗（pandas）
- 使用 `pandas.read_csv()` 读取 CSV 文件并解析日期。
- 使用 `fillna()` 填充缺失值。
- 提取月份周期字段便于分组统计。
- 重新计算 `Total = Quantity * Price`。

In [ ]:
# 读取 CSV 文件
df = pd.read_csv('sales_data.csv', parse_dates=['Date'], dayfirst=True)

# 填充缺失值
df.fillna({'Product': 'Unknown', 'Quantity': 0, 'Price': 0.0, 'Total': 0.0}, inplace=True)

# 保存原始日期列
df['Date_original'] = df['Date']

# 提取月份周期
df['Month'] = df['Date_original'].dt.to_period('M')

# 重新计算总额
df['Total'] = df['Quantity'] * df['Price']

df.head()

## 📈 3. 可视化（matplotlib）
- 使用 `groupby()` 聚合数据并生成图表。
- 使用 `plt.show()` 显示图像。

In [ ]:
# 产品销量柱状图
product_sales = df.groupby('Product')['Quantity'].sum()
product_sales.plot(kind='bar', title='Product Sales Distribution', ylabel='Total Quantity Sold', xlabel='Product')
plt.show()

# 月度销售趋势折线图
monthly_sales = df.groupby('Month')['Total'].sum()
monthly_sales.plot(kind='line', title='Total Sales Over Time', ylabel='Total Sales', xlabel='Month')
plt.show()

## 🗄️ 4. 数据库存储与查询（sqlite3）
- 连接数据库文件并建表。
- 插入清洗后的数据。
- 查询 2023 年总销售额及各产品销量。

In [ ]:
conn = sqlite3.connect('SalesDB.db')
cursor = conn.cursor()

# 建表
cursor.execute('''CREATE TABLE IF NOT EXISTS Sales (
    Date TEXT,
    Product TEXT,
    Quantity INTEGER,
    Price REAL,
    Total REAL
)''')

# 插入数据
for _, row in df.iterrows():
    cursor.execute('INSERT OR IGNORE INTO Sales (Date, Product, Quantity, Price, Total) VALUES (?, ?, ?, ?, ?)',
                   (row['Date'], row['Product'], row['Quantity'], row['Price'], row['Total']))
conn.commit()

# 查询
cursor.execute("SELECT SUM(Total) FROM Sales WHERE strftime('%Y', Date)='2023'")
print('Total Sales in 2023:', cursor.fetchone()[0])

cursor.execute('''
    SELECT Product, SUM(Quantity) FROM Sales
    WHERE strftime('%Y', Date)='2023'
    GROUP BY Product
    ORDER BY SUM(Quantity) DESC
''')
for product, total_quantity in cursor.fetchall():
    print(product, total_quantity)

conn.close()

## 🤖 5. 神经网络实现与训练（PyTorch）
- 定义 `SimpleNN` 三层全连接网络。
- 使用 `MSELoss` 损失函数与 `SGD` 优化器。
- 模拟数据进行训练并绘制损失曲线。

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 3)
        self.fc3 = nn.Linear(3, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# 实例化模型与优化器
model = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 生成模拟数据
data = torch.randn(100, 10)
target = torch.randn(100, 1)
losses = []

# 训练循环
for epoch in range(20):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

# 绘制损失曲线
plt.plot(losses)
plt.title('Training Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()